In [3]:
import warnings 
warnings.filterwarnings("ignore")

In [2]:
from google import genai
from google.genai import types
import base64
import os

In [1]:
from google import genai
from google.genai import types

client = genai.Client(
    vertexai=True,
    project="lending-partner",
    location="asia-south1"
)

rag_tool = types.Tool(
    retrieval=types.Retrieval(
        vertex_rag_store=types.VertexRagStore(
            rag_resources=[
                types.VertexRagStoreRagResource(
                    rag_corpus=(
                        "projects/lending-partner/"
                        "locations/asia-south1/"
                        "ragCorpora/6917529027641081856"
                    )
                )
            ]
        )
    )
)

# config
generate_config = types.GenerateContentConfig(
    temperature=0.3,
    max_output_tokens=2048,
    tools=[rag_tool],
    system_instruction=(
        "You are a financial compliance expert in India. "
        "Answer strictly based on RBI regulations and retrieved documents. "
        "If information is missing from the corpus, say 'Not found in provided documents'. "
        "Do not hallucinate."
    ),
)

# response
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types.Content(
            role="user",
            parts=[
                types.Part(text="What are the rules to issue a credit card?")
            ],
        )
    ],
    config=generate_config,
)

print(response.text)
print("+" * 50)

candidate = response.candidates[0]
grounding = candidate.grounding_metadata

if grounding and grounding.grounding_chunks:
    for chunk in grounding.grounding_chunks:
        print("SOURCE DOCUMENT:")
        print(chunk.retrieved_context.uri)
        print("\nUSED TEXT:")
        print(chunk.retrieved_context.text)
        print("=" * 60)
else:
    print("No grounding chunks retrieved.")


In India, the rules for issuing a credit card are as follows:

**Eligibility of Card-Issuers:**
*   Banks with a net worth of ₹100 crore or more are permitted to conduct credit card business, either independently or through tie-ups with other card-issuing banks or NBFCs, with Board approval.
*   Banks wishing to establish separate subsidiaries for credit card business require prior approval from the Reserve Bank of India.

**Board Approved Policy and Procedures:**
*   Each card-issuer must have a well-documented, Board-approved policy for the issuance and conduct of credit cards, consistent with RBI instructions. This policy should be available on the card-issuer's website.
*   The policy must cover aspects such as interest rate ceilings, charges, default reporting to Credit Information Companies (CICs), co-branding arrangements, discretionary blocking procedures, timelines for lost card formalities, and grievance redressal.
*   Card-issuers must establish a mechanism for half-yearly r

In [7]:
from langchain_classic.memory import ConversationSummaryBufferMemory
from langchain_core.language_models.llms import LLM


c:\Users\Ingit.Paul.in\AppData\Local\anaconda3\envs\review\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ============================================================
# Gemini + Vertex RAG + LangChain Conversation Memory (WORKING)
# ============================================================

from typing import Optional, List
from google import genai
from google.genai import types

# LangChain (CORRECT for your environment)
from langchain_classic.memory import ConversationSummaryBufferMemory
from langchain_core.language_models.llms import LLM


# -------------------------
# 1. Gemini client
# -------------------------
client = genai.Client(
    vertexai=True,
    project="lending-partner",
    location="asia-south1"
)


# -------------------------
# 2. Vertex RAG tool
# -------------------------
rag_tool = types.Tool(
    retrieval=types.Retrieval(
        vertex_rag_store=types.VertexRagStore(
            rag_resources=[
                types.VertexRagStoreRagResource(
                    rag_corpus=(
                        "projects/lending-partner/"
                        "locations/asia-south1/"
                        "ragCorpora/6917529027641081856"
                    )
                )
            ]
        )
    )
)


# -------------------------
# 3. Gemini wrapper for LangChain memory summarization
# -------------------------
class GeminiLLM(LLM):
    client: genai.Client
    model_name: str = "gemini-2.5-flash"

    @property
    def _llm_type(self) -> str:
        return "gemini"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.client.models.generate_content(
            model=self.model_name,
            contents=prompt,
            config={
                "temperature": 0.2,
                "max_output_tokens": 256,
            },
        )
        return response.text


# -------------------------
# 4. LangChain memory
# -------------------------
memory = ConversationSummaryBufferMemory(
    llm=GeminiLLM(client=client),
    max_token_limit=1000,
    memory_key="history",
    input_key="input",
)


# -------------------------
# 5. Static system instruction (IMPORTANT)
# -------------------------
SYSTEM_INSTRUCTION = """
You are a helpful assistant answering questions using retrieved documents.
If the answer is not found in the documents, say so clearly.
"""


# -------------------------
# 6. Chat function (CORRECT payload for Vertex RAG)
# -------------------------
def chat(user_query: str):
    # Load summarized conversation memory
    history_text = memory.load_memory_variables({}).get("history", "")

    contents = []

    # Inject memory as conversational context (NOT system_instruction)
    if history_text.strip():
        contents.append(
            types.Content(
                role="user",
                parts=[
                    types.Part(
                        text=f"Conversation so far:\n{history_text}"
                    )
                ],
            )
        )

    # Current user message
    contents.append(
        types.Content(
            role="user",
            parts=[types.Part(text=user_query)],
        )
    )

    # Gemini call
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents,
        config=types.GenerateContentConfig(
            temperature=0.3,
            max_output_tokens=2048,
            tools=[rag_tool],
            system_instruction=SYSTEM_INSTRUCTION,
        ),
    )

    # Save interaction to LangChain memory
    memory.save_context(
        {"input": user_query},
        {"output": response.text},
    )
    sources = []
    candidate = response.candidates[0]
    grounding = candidate.grounding_metadata

    if grounding and grounding.grounding_chunks:
        for chunk in grounding.grounding_chunks:
            rc = chunk.retrieved_context
            sources.append({
                "uri": rc.uri,
                "text_snippet": rc.text[:300]
            })

    return response.text, sources

In [19]:
chat("What are the rules to issue a credit card?")



ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [11]:
chat("Who issues them?")

('Banks issue credit cards.',
 [{'uri': 'gs://lending-partner/all_documents/155MD.pdf',
   'text_snippet': '(4) Card-issuers shall ensure that loans offered through credit cards are in compliance with \r\nthe instructions on loans and advances issued by the Reserve Bank from time to time. \r\n(5) Card-issuers shall ensure that the credit limit as sanctioned and advised to the \r\ncardholder is not breached at a'},
  {'uri': 'gs://lending-partner/all_documents/155MD.pdf',
   'text_snippet': '(5) Cardholder is a person to whom a card is issued or one who is authorized to use an \r\nissued card. \r\n(6) Card-issuer is a bank which issue debit or credit cards.\r\n(7) Card Loyalty / Reward Programme/s are those schemes linked to a credit card or \r\ndebit card whereby the card-issuer or associated '},
  {'uri': 'gs://lending-partner/all_documents/155MD.pdf',
   'text_snippet': 'Provided that the issuance of a credit card for renewal or replacement shall not be \r\ntreated as an unsolicited 

In [7]:
chat("Does this apply to foreign banks?")

'The provided documents do not specify whether these rules apply to foreign banks. They generally refer to "Banks with net worth of ₹100 crore and above" as eligible to issue credit cards.'

In [8]:
print(memory.buffer)

Human: What are the rules to issue a credit card?
AI: To issue a credit card, banks must have a net worth of ₹100 crore or more and obtain Board approval to undertake credit card business, either independently or through tie-ups. Establishing separate subsidiaries requires prior approval from the Reserve Bank.

Each card-issuer must have a Board-approved policy covering aspects like interest rates, charges, reporting defaults to Credit Information Companies, co-branding, procedures for blocking/deactivating cards, timeframes for lost card formalities, and grievance redressal. Credit card operations must be reviewed half-yearly by the Audit Committee of the Board.

During customer acquisition, card-issuers must provide a one-page Key Fact Statement with the application and convey specific reasons for rejection in writing. The Most Important Terms and Conditions (MITC) must be highlighted and sent to customers at acceptance and whenever conditions are modified, preferably in English, Hin

In [ ]:
cool
